In [ ]:
import pandas as pd
import numpy as np 
import time 
import g2o 
import cv2
import os

# data loader

In [ ]:
folder_path='sample_data/Room_back_home-2024-12-29_19-50-26/'
folder_path_img = folder_path+'/Camera'

gyro = pd.read_csv(folder_path+'Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv(folder_path+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)


image_filenames = sorted(os.listdir(folder_path_img))[1:-1]
image_timestamps = [float(filename.split('.')[0]) for filename in image_filenames] 

def synchronize_data(imu_data, image_timestamps):
    synchronized_data = []
    for img_time in image_timestamps:
        # Find closest IMU timestamp
        closest_imu_index = (np.abs(imu_data['time'] - img_time*1e6)).argmin()
        synchronized_data.append({
            'image_time': img_time,
            'imu_roll': imu_data.iloc[closest_imu_index]['roll'],
            'imu_pitch': imu_data.iloc[closest_imu_index]['pitch'],
            'imu_yaw': imu_data.iloc[closest_imu_index]['yaw'],
            'imu_ax': imu_data.iloc[closest_imu_index]['ax'],
            'imu_ay': imu_data.iloc[closest_imu_index]['ay'],
            'imu_az': imu_data.iloc[closest_imu_index]['az'],
            'image_path': os.path.join(folder_path_img, image_filenames[image_timestamps.index(img_time)])
        })
    return pd.DataFrame(synchronized_data)

synchronized_synchronized_df = synchronize_data(motion, image_timestamps)
synchronized_synchronized_df['time_diff_s'] = synchronized_synchronized_df['image_time'].diff().fillna(0) / 1e3

# keypoint extraction helper functions

In [ ]:
def extract(img):
    """ 
    extract features of an image
    """
    orb = cv2.ORB_create()
 
    # Detection
    pts = cv2.goodFeaturesToTrack(np.mean(img, axis=-1).astype(np.uint8), 1000, qualityLevel=0.01, minDistance=10)
 
    # Extract key points
    kps = [cv2.KeyPoint(f[0][0], f[0][1], 20) for f in pts]
    kps, des = orb.compute(img, kps)
 
    return np.array([(kp.pt[0], kp.pt[1]) for kp in kps]), des

def add_ones(x):
    """ 
    helper function to add ones to have right format of normalized coordinates (TODO: understand better)
    """
    # creates homogenious coordinates given the point x
    return np.concatenate([x, np.ones((x.shape[0], 1))], axis=1)

def normalize(Kinv, pts):
    """
    transform image coordinates to normalized coordinates
    """
    # The inverse camera intrinsic matrix 𝐾^(−1) transforms 2D homogeneous points 
    # from pixel coordinates to normalized image coordinates. 
    return np.dot(Kinv, add_ones(pts).T).T[:, 0:2]

class Frame(object):
    """ 
    Class that has all the frame data
    """
    def __init__(self, img, K, id):
        self.id = id   # unique identifier
        self.K = K     # Intrinsic camera matrix
        self.Kinv = np.linalg.inv(self.K)  # Inverse of the intrinsic camera matrix
 
        pts, self.des = extract(img)             # Extract feature points and descriptors from the image
        self.pts = normalize(self.Kinv, pts)     # Normalizes feature points to normalized coordinates

# keypoint matching
option 1:
- always match against all previous points 
- keep first descriptor pattern OR update descriptor pattern with most recent one
- potential runtime issues -> will explode fast / limit keypoint number
- native loop closure detection

option 2:
- only match against last 3 frames or similar
- how to detect loop closure?

In [ ]:
class UniquePointsFullListMatching(object):
    """ 
    this class tracks all unique points in entire frame, it consists of 3 index parallel lists
    descriptor: list of unique descriptors in entire sequence
    frameIds:   list of frame ids (list of lists), index parallel to descriptor (one descriptor multiple frame ids)
    normPoints: list of normalized points per frame (list of lists), also index parallel to descriptor (one descriptor, multiple frames with a point per frame)
    """
    def __init__(self):
        self.descriptor = [] # (descriptor, [frame_ids])
        self.frameIds   = []
        self.normPoints = [] #normalized point per frame id
        self.initial3dEstimates = []

    def processFrame(self, f):
        if(self.descriptor.empty()):
            for idx, des in enumerate(f.des):
                self.descriptor.append(des)
                self.frameId.append([f.id])
                self.normPoints.append([f.pts[idx]])
        else:
            self.calcAssociations(f)

    def calcAssociations(self, f):
        # The code performs k-nearest neighbors matching on feature descriptors
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        matches = bf.knnMatch(np.array(self.descriptor), f.des, k=2)    

        # applies Lowe's ratio test to filter out good 
        # matches based on a distance threshold.
        ret = []
        idx1, idx2 = [], []
        for m, n in matches:
            if m.distance < 0.75*n.distance:
                p1 = self.normPoints[m.queryIdx].end()  # take last measured point as most recent to determine l2 norm
                p2 = f.pts[m.trainIdx]

                # Distance test
                # dditional distance test, ensuring that the 
                # Euclidean distance between p1 and p2 is less than 0.1
                if np.linalg.norm((p1-p2)) < 0.1:
                    self.associate(m.queryIdx, f.id, p2)
                    pass  

    def associate(self, uniquePtIdx, frameId, newPoint):
        self.frameIds[uniquePtIdx].append(frameId)  # add new frameid to associated point
        self.normPoints[uniquePtIdx].append(newPoint)  # add new normalized point to associated point

    def initial3dEstimate(self):
        #TODO, maybe not needed, 
        # do the initial easy triangulation (optimization problem)
        # can only be done for unique points with more than 1 frame


    # Idea: in a for loop: 
    # - track unique points: (descriptor, [frame_ids])
    # - when processing new frame we check if we have matches to the unique points
    # - if match to a unique point -> append frame id to unique point
    # - otherwise append new unique point to list

    # -> disadvantage (TODO), how do i update the descriptors over multiple frames? maybe not necessary
    # -> maybe possible

In [ ]:
# run association
uniquePts = UniquePointsFullListMatching()
..

# setup graph
- set poses as nodes (TODO: how to define edges between imu data/poses)
- create characteristic points per frame
- find matches between current frame and all other frames
- add edges for common points

Potential follow ups:
- add another pose estimate through image data + edge?

In [ ]:
# Sets optimizer 
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)


K = np.array([[730, 0, 620], [0, 730, 360], [0, 0, 1]])  # Example values
g2o.VertexSCam.set_cam(*K)

# Create vertices from cameras
for idx_extr, extrinsic in enumerate(extrinsics):
    pose = g2o.Isometry3d(np.linalg.inv(extrinsic))
    v_se3 = g2o.VertexSCam()
    v_se3.set_id(idx_extr)
    v_se3.set_estimate(pose)
    v_se3.set_fixed(False)
    v_se3.set_all() # Calculates matrices related to projection
    optimizer.add_vertex(v_se3)

# Each vertex must have a unique id. 
# Ensure point and camera ids do not overlap.
last_pose_id = idx_extr
first_point_id = last_pose_id + 1

# Create vertices from all points and connect them to the cameras
for idx, des in enumerate(uniquePts.descriptor):
    point_id = idx + first_point_id
    
    # Create a point vertex with distorted estimate
    vertex_point = g2o.VertexSBAPointXYZ()
    vertex_point.set_id(point_id)
    # see https://github.com/RainerKuemmerle/g2o/issues/109 on why we use this
    vertex_point.set_marginalized(True)  
    vertex_point.set_estimate(distorted_point) #I guess estimated 3d position, maybe I can set it to 1,0,0 or pose estimate
    optimizer.add_vertex(vertex_point)
    
    # Create an point-camera edge with each camera
    for idxAssocPt, projected_point in enumerate(uniquePts.normPoints[idx]):
        frameId = uniquePts.frameIds[idx][idxAssocPt] # from the current unique point get the frame id that we're just processing
        cameraPoseId = frameId # need to ensure this is true
        edge = g2o.Edge_XYZ_VSC()
        edge.set_vertex(0, vertex_point) 
        edge.set_vertex(1, optimizer.vertex(cameraPoseId))
        edge.set_measurement(projected_point) 
        # The information matrix is the measurement 
        # uncertainty. We set this equal for all measurements.
        edge.set_information(np.identity(3))
        edge.set_parameter_id(0, 0)
        optimizer.add_edge(edge)
    last_point_id = point_id

optimizer.initialize_optimization()
optimizer.set_verbose(False)
print("g2o graph has been created") 